# Cost Allocation from Constraint Matrix

### Import packages

In [246]:
import pypsa
import pandas as pd
from helpers import get_linear_system, noisy_lopf
import networks

## Load example network

### One Bus, One Snapshot, Two Generators without invesment

$G_1 < d < G_1 + G_2$

Note, for such a system the total system cost $TC$ are less then the nodal payments as soon as one generator is at its limit: 

$\lambda = o_s + \bar{\mu_s} \;\;\;\;\; \forall s$

$d \, \lambda = d \, (o_s + \bar{\mu_s} ) \ge \sum_s g_s \, o_s = TC$

In [260]:
n = networks.n1_t1_g2_wo()
noisy_lopf(n)

s = get_linear_system(n)
A_, A_inv, c, x, d, m, r = (s[k].round(2) for k in ['A_', 'A_inv', 'c', 'x', 'd', 'm', 'r'])

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.11s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Read LP format model from file /tmp/pypsa-problem-qx7uzjfb.lp
Reading time = 0.00 seconds
obj: 5 rows, 3 columns, 6 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 5 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xd6148c94
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 7e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9999977e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.999997650e+02


INFO:pypsa.linopf:Optimization successful. Objective value: 5.00e+02


In [292]:
n.loads_t.p

,0
now,70.0


In [293]:
n.generators_t.p

,Gen0,Gen1
now,40.000047,29.999953


In [294]:
n.generators[['p_nom', 'p_nom_extendable', 'p_nom_opt', 'marginal_cost', 'capital_cost']]

,p_nom,p_nom_extendable,p_nom_opt,marginal_cost,capital_cost
Gen0,40.0,False,40.0,5.0,0.0
Gen1,40.0,False,40.0,10.0,0.0


# Understand the Cost Allocation 


Summerizing equations:

$\sum_i x_i \, A'_{i,j} = d_j \hspace{10pt} \leftrightarrow \hspace{10pt} x_i = \sum_j {A'}_{i,j}^{-1} \, d_j$

$\sum_j A'_{i,j} \, \mu_j = c_i \hspace{10pt} \leftrightarrow \hspace{10pt} \mu_j = \sum_i c_i \, {A'}_{i,j}^{-1}$

The total cost can be represent through all of the following

$TC = \sum_i c_i \, x_i = \sum_{i,j}  c_i \, {A'}^{-1}_{ij} d_j = \sum_{i,j} x_i \, A'_{i,j} \, \mu_j = \sum_j \mu_j \, d_j $

In [296]:
assert all(A_.T @ x == d)

assert all(A_ @ m == c)

assert round(n.objective, 2) ==  A_inv @ d @ c == A_ @ m @ x

The basis of the cost allocation is ${A'}^{-1}$. It connects binding constraint to the variables

$x_i = \sum_j {A'}^{-1}_{i,j} \, d_j$

If ${A'}^{-1}_{i,j}$ is positive, $d_j$ has a positive effect on $x_i$. If negative, $d_j$ pushes $x_i$ down. 

In [297]:
A_inv * d

component                           Generator            Bus
name                                 mu_upper marginal_price
component_i                              Gen0           Bus1
snapshot                                  now            now
component name component_i snapshot                         
Generator p    Gen0        now           40.0            0.0
               Gen1        now          -40.0           70.0

The same counts for the cost of the variables $C_{i,j}$ defined as  

$C_{ij} = {A'}^{-1}_{ij} \, c_i \, d_j$

If $C_{i,j}$ is positive, constraint $j$ pushes expences for variables $i$ up, if negative it lowers them. 

In [262]:
C = (A_inv * d).mul(c, 0)
assert round(n.objective, 2) ==  C.sum().sum()
C

component                           Generator            Bus
name                                 mu_upper marginal_price
component_i                              Gen0           Bus1
snapshot                                  now            now
component name component_i snapshot                         
Generator p    Gen0        now          200.0            0.0
               Gen1        now         -400.0          700.0

In [266]:
C.sum().to_frame('Constraint induced cost')

,,,,Constraint induced cost
component,name,component_i,snapshot,
Generator,mu_upper,Gen0,now,-200.0
Bus,marginal_price,Bus1,now,700.0


In [267]:
C.sum(1).to_frame('Cost per variable')

Cost per variable
component name component_i snapshot                   
Generator p    Gen0        now                   200.0
               Gen1        now                   300.0

### Other direction

The matrix $A'_{i,j}$ connects to the variables their binding constraints

$\sum_i x_i \, A'_{i,j} = d_j$


In [268]:
A_.mul(x, 0)

component                           Generator            Bus
name                                 mu_upper marginal_price
component_i                              Gen0           Bus1
snapshot                                  now            now
component name component_i snapshot                         
Generator p    Gen0        now           40.0           40.0
               Gen1        now            0.0           30.0

In [273]:
D = (A_ * m).mul(x, 0) 
assert round(n.objective, 2) ==  D.sum().sum()
D

component                           Generator            Bus
name                                 mu_upper marginal_price
component_i                              Gen0           Bus1
snapshot                                  now            now
component name component_i snapshot                         
Generator p    Gen0        now         -200.0          400.0
               Gen1        now           -0.0          300.0